In [6]:
import random
import math
from mesa.space import MultiGrid
from mesa import Agent, Model

# --- Custom AgentList with shuffle_do ---
class AgentList:
    def __init__(self):
        self.agents = []

    def add(self, agent):
        self.agents.append(agent)

    def remove(self, agent):
        if agent in self.agents:
            self.agents.remove(agent)

    def shuffle_do(self, method_name):
        # Shuffle the list and call the given method on each agent.
        random.shuffle(self.agents)
        for agent in list(self.agents):
            getattr(agent, method_name)()

    def __iter__(self):
        return iter(self.agents)

# --- Fuzzy Cognitive Map (FCM) Model ---
class FCM:
    def __init__(self, concepts, weights):
        """
        concepts: dict mapping concept names to activation values.
        weights: dict mapping (from_concept, to_concept) to weight values.
        """
        self.concepts = concepts
        self.weights = weights

    def update(self, perception):
        """
        Update activation levels.
        Sensitive concepts get updated directly from perception,
        and other concepts are updated via a weighted sum.
        """
        new_values = {}
        for concept in self.concepts:
            if concept in perception:
                new_values[concept] = perception[concept]
            else:
                influence = sum(self.concepts[other] * self.weights.get((other, concept), 0)
                                for other in self.concepts)
                new_values[concept] = self.sigmoid(influence)
        self.concepts = new_values

    def sigmoid(self, x):
        return 1 / (1 + math.exp(-x))

    def genome(self):
        """Return a copy of the genome (weights)."""
        return self.weights.copy()

    def copy(self):
        return FCM(self.concepts.copy(), self.weights.copy())

# --- Genome utilities: mutation and distance ---
def mutate_genome(genome, mutation_rate=0.01, mutation_strength=0.1):
    new_genome = {}
    for key, weight in genome.items():
        if random.random() < mutation_rate:
            new_genome[key] = weight + random.uniform(-mutation_strength, mutation_strength)
        else:
            new_genome[key] = weight
    return new_genome

def genome_distance(genome1, genome2):
    all_keys = set(genome1.keys()).union(set(genome2.keys()))
    distance = sum(abs(genome1.get(key, 0) - genome2.get(key, 0)) for key in all_keys)
    return distance

# --- Ecosystem Agent ---
class EcosystemAgent(Agent):
    def __init__(self, unique_id, model, agent_type, fcm, energy, species):
        super().__init__(unique_id, model)
        self.agent_type = agent_type  # 'prey' or 'predator'
        self.fcm = fcm
        self.energy = energy
        self.species = species
        self.age = 0
        self.reproduction_threshold = 20 if agent_type == 'prey' else 30

        # Register self with the model's custom agent list.
        self.model.custom_agents.add(self)

    def perceive(self):
        """Simplified perception based on agent type."""
        perception = {}
        if self.agent_type == 'prey':
            perception['predator_distance'] = random.uniform(0, 1)
            perception['food_level'] = self.model.get_cell_food(self.pos)
        else:
            perception['prey_distance'] = random.uniform(0, 1)
            perception['meat_level'] = self.model.get_cell_meat(self.pos)
        return perception

    def decide_action(self):
        """Choose an action based on FCM activations."""
        if self.agent_type == 'prey':
            actions = ['evade', 'eat', 'reproduce', 'move']
        else:
            actions = ['hunt', 'eat', 'reproduce', 'move']
        weights = [self.fcm.concepts.get(a, random.random()) for a in actions]
        total = sum(weights)
        if total == 0:
            return random.choice(actions)
        r = random.uniform(0, total)
        cumulative = 0
        for action, weight in zip(actions, weights):
            cumulative += weight
            if r <= cumulative:
                return action
        return actions[-1]

    def step(self):
        perception = self.perceive()
        self.fcm.update(perception)
        action = self.decide_action()
        if action in ['evade', 'move']:
            self.move()
        elif action == 'eat':
            self.eat()
        elif action == 'hunt':
            self.hunt()
        elif action == 'reproduce':
            self.reproduce()
        self.energy -= 1
        self.age += 1
        if self.energy <= 0 or self.age > 50:
            self.die()

    def move(self):
        possible_steps = self.model.grid.get_neighborhood(self.pos, moore=True, include_center=False)
        if possible_steps:
            new_pos = random.choice(possible_steps)
            self.model.grid.move_agent(self, new_pos)

    def eat(self):
        if self.agent_type == 'prey':
            food = self.model.consume_food(self.pos)
            self.energy += food
        else:
            meat = self.model.consume_meat(self.pos)
            self.energy += meat

    def hunt(self):
        cellmates = self.model.grid.get_cell_list_contents([self.pos])
        for other in cellmates:
            if other.agent_type == 'prey':
                other.die()
                self.energy += 5
                break

    def reproduce(self):
        if self.energy >= self.reproduction_threshold:
            cellmates = self.model.grid.get_cell_list_contents([self.pos])
            mates = [other for other in cellmates
                     if other.agent_type == self.agent_type and other != self
                     and other.energy >= self.reproduction_threshold]
            if mates:
                mate = random.choice(mates)
                self.interbreed(mate)

    def interbreed(self, mate):
        offspring_genome = {}
        keys = set(list(self.fcm.weights.keys()) + list(mate.fcm.weights.keys()))
        for key in keys:
            chosen = self.fcm.weights.get(key, 0) if random.random() < 0.5 else mate.fcm.weights.get(key, 0)
            offspring_genome[key] = chosen
        offspring_genome = mutate_genome(offspring_genome)
        offspring_concepts = {c: random.random() for c in self.fcm.concepts.keys()}
        offspring_fcm = FCM(offspring_concepts, offspring_genome)
        dist = genome_distance(offspring_fcm.genome(), self.fcm.genome())
        new_species = self.species
        threshold = 0.5
        if dist > threshold:
            new_species = self.model.get_new_species_id()
        offspring = EcosystemAgent(self.model.next_id(), self.model, self.agent_type, offspring_fcm, energy=10, species=new_species)
        self.model.grid.place_agent(offspring, self.pos)
        self.energy -= self.reproduction_threshold // 2
        mate.energy -= mate.reproduction_threshold // 2

    def die(self):
        self.model.grid.remove_agent(self)
        self.model.custom_agents.remove(self)

# --- Ecosystem Model ---
class EcosystemModel(Model):
    def __init__(self, width, height, initial_predators, initial_preys):
        self.width = width
        self.height = height
        self.grid = MultiGrid(width, height, torus=True)
        self.custom_agents = AgentList()  # Custom storage for agents.
        self.species_counter = 0
        self.unique_id = 0

        # Initialize resources in each cell.
        self.resources = {}
        for x in range(width):
            for y in range(height):
                self.resources[(x, y)] = {'food': random.randint(0, 3), 'meat': 0}

        # Create initial prey agents.
        for _ in range(initial_preys):
            fcm = FCM(
                {'hunger': random.random(), 'fear': random.random(), 'reproduce': random.random()},
                {('hunger', 'reproduce'): random.uniform(-1, 1), ('fear', 'reproduce'): random.uniform(-1, 1)}
            )
            agent = EcosystemAgent(self.next_id(), self, 'prey', fcm, energy=10, species=self.get_new_species_id())
            x = random.randrange(width)
            y = random.randrange(height)
            self.grid.place_agent(agent, (x, y))
        # Create initial predator agents.
        for _ in range(initial_predators):
            fcm = FCM(
                {'hunger': random.random(), 'aggression': random.random(), 'reproduce': random.random()},
                {('hunger', 'reproduce'): random.uniform(-1, 1), ('aggression', 'reproduce'): random.uniform(-1, 1)}
            )
            agent = EcosystemAgent(self.next_id(), self, 'predator', fcm, energy=15, species=self.get_new_species_id())
            x = random.randrange(width)
            y = random.randrange(height)
            self.grid.place_agent(agent, (x, y))
        self.running = True

    def next_id(self):
        self.unique_id += 1
        return self.unique_id

    def get_new_species_id(self):
        self.species_counter += 1
        return self.species_counter

    def get_cell_food(self, pos):
        return self.resources[pos]['food']

    def get_cell_meat(self, pos):
        return self.resources[pos]['meat']

    def consume_food(self, pos):
        if self.resources[pos]['food'] > 0:
            self.resources[pos]['food'] -= 1
            return 3  # Energy gained.
        return 0

    def consume_meat(self, pos):
        if self.resources[pos]['meat'] > 0:
            self.resources[pos]['meat'] -= 1
            return 5  # Energy gained.
        return 0

    def step(self):
        # Update resources.
        for pos in self.resources:
            self.resources[pos]['food'] = min(self.resources[pos]['food'] + random.choice([0, 1]), 3)
            if self.resources[pos]['meat'] > 0:
                self.resources[pos]['meat'] = max(self.resources[pos]['meat'] - 1, 0)
        # Shuffle custom agents and execute their step method.
        self.custom_agents.shuffle_do("step")

model = EcosystemModel(width=10, height=10, initial_predators=5, initial_preys=20)
for i in range(100):
    model.step()



TypeError: object.__init__() takes exactly one argument (the instance to initialize)